## データの読込

In [1]:
import pandas as pd
df = pd.read_csv("vgsales-12-4-2019.csv")
df.head(3)

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg


In [2]:
import time
import random
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from natsort import natsorted 

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from autogluon.tabular import TabularPredictor

import warnings
warnings.simplefilter('ignore')

ModuleNotFoundError: No module named 'autogluon'

In [ ]:
# シード値の固定
SEED = 42

random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [ ]:
# 目的変数にNull値があっても学習・予測できるが、評価時にエラーが出るのでNull値を補完する

df["Global_Sales"] = df["Global_Sales"].fillna(df["Global_Sales"].mean())

In [ ]:
train, test = train_test_split(
                            df.drop(["NA_Sales", "PAL_Sales", "JP_Sales", "Other_Sales"], axis=1), 
                            test_size=0.3,
                            shuffle=True, 
                            random_state=SEED
                        )

## AutoGluon

In [ ]:
# 学習・推論
gkf = GroupKFold(n_splits=5)

groups = train["Genre"]

cv_result_gln_eval = []

for i, (train_index, test_index) in enumerate(gkf.split(train, train, groups)):
    train_gkf, test_gkf = train.iloc[train_index], train.iloc[test_index]
    
    print(f"Fold {i}")

    model = TabularPredictor(
                    label = "Global_Sales",                  # 目的変数
                    problem_type = "regression",             # 回帰を指定
                    eval_metric = "root_mean_squared_error", # 評価指標
                    verbosity = 0,                           # ログの非表示
                )

    # 学習
    model.fit(
            train_data=train_gkf,
            ag_args_fit={'num_gpus': 1}, # GPUの使用
            time_limit = 30,             # 学習時間の上限
            )

    # 予測
    y_pred = model.predict(test_gkf.drop(["Global_Sales"], axis=1))

    # AutoGluon の解釈性機能
    print("特徴量の重要度")
    feature_importance_df = model.feature_importance(
                                test_gkf,
                                silent = True
                            )

    display(feature_importance_df)
    
    # 特徴量の重要度をグラフ表示
    plt.figure(figsize=(12, 8))
    plt.title(f"Fold {i} 特徴量の重要度")
    plt.xlabel("重要度")
    plt.ylabel("特徴量")
    
    plt.barh(
        feature_importance_df.index,
        feature_importance_df["importance"],
        )
    plt.show()
    print()

    # AutoGluon の機能
    print("リーダーボード")
    display(
        model.leaderboard(
            test_gkf,
            silent = True
        )
    )
    print()

    # AutoGluon の評価機能
    score = model.evaluate_predictions(y_true=test_gkf["Global_Sales"], y_pred=y_pred)

    rmse_gln_eval = score['root_mean_squared_error']

    cv_result_gln_eval.append(rmse_gln_eval)

print("RMSE:", np.mean(cv_result_skl_eval))

In [ ]:
print("RMSE:", round(np.mean(cv_result_skl_eval), 3))